In [2]:
from binance.client import Client
from binance.enums import *
from keras.engine import sequential
from keras.layers.recurrent import RNN
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, LSTM, Dropout, Bidirectional, GRU, ConvLSTM2D, Activation, CuDNNGRU, CuDNNLSTM, Conv2D, Conv1D, Input, MaxPooling1D, \
                         GlobalMaxPooling1D, Embedding
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, \
                                       EarlyStopping, LearningRateScheduler, \
                                       ReduceLROnPlateau, CSVLogger
from tensorflow.keras.callbacks.experimental import BackupAndRestore
from tensorflow.keras.metrics import Accuracy, BinaryAccuracy, \
                                     CategoricalAccuracy, \
                                     SparseCategoricalAccuracy, \
                                     TopKCategoricalAccuracy, \
                                     SparseTopKCategoricalAccuracy

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import math
import keras
import sys 
import pandas as pd
import numpy as np
import json
import os

2021-11-08 20:15:58.037544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-08 20:15:58.037587: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Pooling Layer
Convolutional layers in a convolutional neural network summarize the presence of features in an input image.<br>

A problem with the output feature maps is that they are sensitive to the location of the features in the input. One approach to address this sensitivity is to down sample the feature maps. This has the effect of making the resulting down sampled feature maps more robust to changes in the position of the feature in the image, referred to by the technical phrase “local translation invariance.”<br>

Pooling layers provide an approach to down sampling feature maps by summarizing the presence of features in patches of the feature map. Two common pooling methods are average pooling and max pooling that summarize the average presence of a feature and the most activated presence of a feature respectively.
- **Average Pooling:** Calculate the average value for each patch on the feature map.
- **Maximum Pooling (or Max Pooling):** Calculate the maximum value for each patch of the feature map.

The result of using a pooling layer and creating down sampled or pooled feature maps is a summarized version of the features detected in the input. They are useful as small changes in the location of the feature in the input detected by the convolutional layer will result in a pooled feature map with the feature in the same location. This capability added by pooling is called the model’s invariance to local translation.

<center><img src="assets/pooling.png"></img></center>

# MaxPooling1D Layer
Max pooling operation for 1D temporal data.<br>

Downsamples the input representation by taking the maximum value over a spatial window of size pool_size. The window is shifted by strides. The resulting output, when using the "valid" padding option, has a shape of: output_shape = (input_shape - pool_size + 1) / strides)<br>

The resulting output shape when using the "same" padding option is: output_shape = input_shape / strides<br>

In [3]:
tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=None, padding="valid", data_format="channels_last"
)

## Arguments
- **pool_size:** Integer, size of the max pooling window.
- **strides:** Integer, or None. Specifies how much the pooling window moves for each pooling step. If None, it will default to pool_size.
- **padding:** One of "valid" or "same" (case-insensitive). "valid" means no padding. "same" results in padding evenly to the left/right or up/down of the input such that output has the same height/width dimension as the input.
- **data_format:** A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. channels_last corresponds to inputs with shape (batch, steps, features) while channels_first corresponds to inputs with shape (batch, features, steps).

## Input Shape
- If data_format='channels_last': 3D tensor with shape (batch_size, steps, features).
- If data_format='channels_first': 3D tensor with shape (batch_size, features, steps).

## Output Shape
- If data_format='channels_last': 3D tensor with shape (batch_size, downsampled_steps, features).
- If data_format='channels_first': 3D tensor with shape (batch_size, features, downsampled_steps).

## Example
Maximum pooling, or max pooling, is a pooling operation that calculates the maximum, or largest, value in each patch of each feature map.<br>

The results are down sampled or pooled feature maps that highlight the most present feature in the patch, not the average presence of the feature in the case of average pooling. This has been found to work better in practice than average pooling for computer vision tasks like image classification.<br>

We can make the max pooling operation concrete by again applying it to the output feature map of the line detector convolutional operation and manually calculate the first row of the pooled feature map.

In [6]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
# define input data
data = [[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0]]
data = asarray(data)
data = data.reshape(1, 8, 8, 1)
# create model
model = Sequential()
model.add(Conv2D(1, (3,3), activation='relu', input_shape=(8, 8, 1)))
model.add(MaxPooling2D())
# summarize model
model.summary()
# define a vertical line detector
detector = [[[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]]]
weights = [asarray(detector), asarray([0.0])]
# store the weights in the model
model.set_weights(weights)
# apply filter to input data
yhat = model.predict(data)
# enumerate rows
for r in range(yhat.shape[1]):
	# print each column in the row
	print([yhat[0,r,c,0] for c in range(yhat.shape[2])])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 6, 6, 1)           10        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 1)           0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[0.0, 3.0, 0.0]
[0.0, 3.0, 0.0]
[0.0, 3.0, 0.0]


# AveragePooling1D Layer
Average pooling for temporal data.<br>

Downsamples the input representation by taking the average value over the window defined by pool_size. The window is shifted by strides. The resulting output when using "valid" padding option has a shape of: output_shape = (input_shape - pool_size + 1) / strides)<br>

The resulting output shape when using the "same" padding option is: output_shape = input_shape / strides<br>

In [4]:
tf.keras.layers.AveragePooling1D(
    pool_size=2, strides=None, padding="valid", data_format="channels_last"
)

## Arguments
- **pool_size:** Integer, size of the average pooling windows.
- **strides:** Integer, or None. Factor by which to downscale. E.g. 2 will halve the input. If None, it will default to pool_size.
- **padding:** One of "valid" or "same" (case-insensitive). "valid" means no padding. "same" results in padding evenly to the left/right or up/down of the input such that output has the same height/width dimension as the input.
- **data_format:** A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. channels_last corresponds to inputs with shape (batch, steps, features) while channels_first corresponds to inputs with shape (batch, features, steps).

## Input shape
- If data_format='channels_last': 3D tensor with shape (batch_size, steps, features).
- If data_format='channels_first': 3D tensor with shape (batch_size, features, steps).

## Output shape
- If data_format='channels_last': 3D tensor with shape (batch_size, downsampled_steps, features).
- If data_format='channels_first': 3D tensor with shape (batch_size, features, downsampled_steps).

## Example
On two-dimensional feature maps, pooling is typically applied in 2×2 patches of the feature map with a stride of (2,2).<br>

Average pooling involves calculating the average for each patch of the feature map. This means that each 2×2 square of the feature map is down sampled to the average value in the square.<br>

For example, the output of the line detector convolutional filter in the previous section was a 6×6 feature map. We can look at applying the average pooling operation to the first line of that feature map manually.<br>

In [5]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D
# define input data
data = [[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0]]
data = asarray(data)
data = data.reshape(1, 8, 8, 1)
# create model
model = Sequential()
model.add(Conv2D(1, (3,3), activation='relu', input_shape=(8, 8, 1)))
model.add(AveragePooling2D())
# summarize model
model.summary()
# define a vertical line detector
detector = [[[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]]]
weights = [asarray(detector), asarray([0.0])]
# store the weights in the model
model.set_weights(weights)
# apply filter to input data
yhat = model.predict(data)
# enumerate rows
for r in range(yhat.shape[1]):
	# print each column in the row
	print([yhat[0,r,c,0] for c in range(yhat.shape[2])])

2021-11-08 20:23:29.560582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-08 20:23:29.560636: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-08 20:23:29.560665: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aspire): /proc/driver/nvidia/version does not exist
2021-11-08 20:23:29.561915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-08 20:23:29.714140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimiz

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 6, 1)           10        
_________________________________________________________________
average_pooling2d (AveragePo (None, 3, 3, 1)           0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[0.0, 3.0, 0.0]
[0.0, 3.0, 0.0]
[0.0, 3.0, 0.0]


# GlobalPooling1D Layer
Downsamples the input representation by taking the maximum value over the time dimension.

Instead of down sampling patches of the input feature map, global pooling down samples the entire feature map to a single value. This would be the same as setting the pool_size to the size of the input feature map.<br>

Global pooling can be used in a model to aggressively summarize the presence of a feature in an image. It is also sometimes used in models as an alternative to using a fully connected layer to transition from feature maps to an output prediction for the model.<br>

Both global average pooling and global max pooling are supported by Keras via the GlobalAveragePooling2D and GlobalMaxPooling2D classes respectively.<br>

In [7]:
tf.keras.layers.GlobalMaxPooling1D(
    data_format="channels_last", keepdims=False
)

## Arguments
- **data_format:** A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. channels_last corresponds to inputs with shape (batch, steps, features) while channels_first corresponds to inputs with shape (batch, features, steps).
- **keepdims:** A boolean, whether to keep the temporal dimension or not. If keepdims is False (default), the rank of the tensor is reduced for spatial dimensions. If keepdims is True, the temporal dimension are retained with length 1. The behavior is the same as for tf.reduce_max or np.max.

## Input Shape
- If data_format='channels_last': 3D tensor with shape: (batch_size, steps, features)
- If data_format='channels_first': 3D tensor with shape: (batch_size, features, steps)

## Output Shape
- If keepdims=False: 2D tensor with shape (batch_size, features).
- If keepdims=True:
    - If data_format='channels_last': 3D tensor with shape (batch_size, 1, features)
    - If data_format='channels_first': 3D tensor with shape (batch_size, features, 1)

## Example

In [8]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import GlobalMaxPooling2D
# define input data
data = [[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0],
		[0, 0, 0, 1, 1, 0, 0, 0]]
data = asarray(data)
data = data.reshape(1, 8, 8, 1)
# create model
model = Sequential()
model.add(Conv2D(1, (3,3), activation='relu', input_shape=(8, 8, 1)))
model.add(GlobalMaxPooling2D())
# summarize model
model.summary()
# # define a vertical line detector
detector = [[[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]]]
weights = [asarray(detector), asarray([0.0])]
# store the weights in the model
model.set_weights(weights)
# apply filter to input data
yhat = model.predict(data)
# enumerate rows
print(yhat)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 6, 6, 1)           10        
_________________________________________________________________
global_max_pooling2d (Global (None, 1)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[[3.]]
